# General Usage

In [10]:
%load_ext autoreload
%autoreload 2
%pprint off

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Pretty printing has been turned OFF


In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
%pprint

Pretty printing has been turned ON


### Preamble

[!] use only if you have **NOT** installed FHIR PACK via `pip`, `pipenv` or similar in your environment but you have instead cloned the project and are now in the `/examples` directory

In [13]:
import sys,os
sys.path.append(os.getcwd()+'/../src/')
# sys.path

### Imports

In [14]:
import fhirpack			as fp
import pandas			as pd
import fhirpy
import numpy			as np
import fhir.resources	as fr
import json
from tqdm import tqdm
from fhirpack.constants import CONFIG

### Useful Paths for Data Storage 

In [15]:
DATAPATH=CONFIG.get('DATAPATH')
RESPATH=f"{DATAPATH}/fhir/"

### FHIR Server Connection

### CLI Usage

In [16]:
!python -m fhirpack.cli -e .env.example "-o" "getPatients" -p "given = Chalmers" #-v

                                                data  Patient                   
0  {'resourceType': 'Patient', 'id': '1567099', '...  1567099


In [17]:
# public FHIR test servers may be unavailable sometimes
# if you prefer to test FHIRPACK using a locally running FHIR server
# you may use our provided Docker environment by changing into the
# cloned repository and running `docker-compose up``

# !fp -e .env.docker -o "getPatients" -p "given = Max" #-v

                                                data Patient                    
0  {'resourceType': 'Patient', 'id': '146053', 'm...  146053
1  {'resourceType': 'Patient', 'id': '146255', 'm...  146255
2  {'resourceType': 'Patient', 'id': '146496', 'm...  146496


### From Custom Environment File

In [18]:
# pack = fp.PACK(envFile='.env.docker')

# for i,e in pack.getPatients(['146053']).data.items():
# 	print(
# 		json.dumps(e.serialize(), indent=4, sort_keys=True)[:200],
# 		'\n...'
# 	)

{
    "address": [
        {
            "city": "Walpole",
            "country": "US",
            "extension": [
                {
                    "extension": [
                        {
      
...


In [19]:
!cat .env.example



SCHEMA=https
PORT=80

#DOMAIN=${SCHEMA}://hapi.fhir.org
#DOMAIN=${SCHEMA}://test.hapifhir.io
#might be unstable or unavailable https://github.com/hapifhir/hapi-fhir/issues/1171#issuecomment-1179728061

DOMAIN=${SCHEMA}://hapi.fhir.org

#DOMAIN=${SCHEMA}://example.com:${PORT}
# only use port if the server is running on a non-standard port 
# and this is reflected in the FHIR pagination links
# see https://gitlab.com/fhirpack/main/-/issues/75 for more information

APIBASE=${DOMAIN}/baseR4

#AUTH_METHOD=
#AUTH_PARAMS_PRESET=
#LOGINURL=${DOMAIN}/oauth/token
#OAUTH_TOKEN_ENDPOINT=${DOMAIN}/oauth/token
#OAUTH_USERNAME=username
#OAUTH_PASSWORD=password
#OAUTH_TOKEN=token

LOGSPATH=./logs
DATAPATH=./data

#EXTRACTION_BASE_TOKEN_DICOM=${OAUTH_TOKEN}



### From Default .env Environment File

In [20]:
# pack = fp.PACK()
# pack.client.authorization

### Manual Setup

In [21]:

# pack  = fp.PACK(envFile=".env.docker")

pack = fp.PACK("https://hapi.fhir.org/baseR4")
pack.getPatients(['2918556'])

# pack  = fp.PACK("http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir")
# pack.getPatients(['146053'])

,data,Patient
0,"{'resourceType': 'Patient', 'id': '2918556', '...",2918556


In [22]:
#testing with another less reliable FHIR server
# pack  = fp.PACK("http://test.fhir.org/r4/")
# pack.getPatients(['11']).data.iloc[0]

### Testing with A Docker FHIR Server and Synthetic Data

In [23]:
# alternatively, if you're using our Dockerfile to run your FHIR server
# remember to run "docker-compose up" in your local clone of FHIRPACK first  

# pack  = fp.PACK(envFile=".env.docker")
# pack  = fp.PACK("http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir")
# pack.getPatients(['1']).data.iloc[0].serialize()
# pack.getPatients(['146053'])


### Introductory Example

In [24]:
# for patients whose lastname is Koepp, find the root identity,
# get their associated diagnostic reports and report their ID and status

pack.getPatients(searchParams={"family":"koepp"})\
	.getDiagnosticReports()\
		.getPatients()\
			.getRootPatients()\
				.gatherSimplePaths(['id','birthDate'])

,id,birthDate
0,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
1,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
2,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
3,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
4,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
...,...,...
64,2744581,1956-08-03
65,2744581,1956-08-03
66,2744581,1956-08-03
67,2744581,1956-08-03


### FHIR PACK Usage 

### References

In [25]:
patientReference = pack.getReferences(['Patient/647487'])
patientReference.data.iloc[0].to_resource().serialize()

{'resourceType': 'Patient',
 'id': '647487',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-03-20T21:38:58.974+00:00',
  'source': '#bE19PhJ3nQMhB1xy'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Series_Status: Not complete Evaluation_Status_1.0: Valid Evaluation_Status_2.0: Valid Evaluation_Status_3.0: Valid Evaluation_Status_4.0: Valid Evaluation_Status_5.0: Valid <b># 5 PENTACEL AT ≥ 4 YRS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>20 January 2016</span></td></tr></tbody></table></div>'},
 'name': [{'family': '# 5 Pentacel at ≥ 4 yrs',
   'given': ['Series_Status: Not complete',
    'Evaluation_Status_1.0: Valid',
    'Evaluation_Status_2.0: Valid',
    'Evaluation_Status_3.0: Valid',
    'Evaluation_Status_4.0: Valid',
    'Evaluation_Status_5.0: Valid']}],
 'gender': 'female',
 'birthDate': '2016-01-20'}

In [26]:
# getResources applied to References simplifies the conversion
# of FHIR/FHIRpy References to Resources

patientReference.getResources()

,data,Patient
0,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Resources

### Reference to Resource

In [27]:
multiPatientReference = pack.getReferences(
    [
        'Patient/1849165',
        'Patient/647487'
    ])

multiPatientReference.data.apply( lambda x:x.id )

0    1849165
1     647487
Name: data, dtype: object

In [28]:
multiPatientReference.getResources()

,data,Patient
0,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165
1,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Direct Resource

In [29]:
pack.getPatients(
	searchParams={"given":"max"}
)

,data,Patient
0,"{'resourceType': 'Patient', 'id': '2583214', '...",2583214
1,"{'resourceType': 'Patient', 'id': '2583215', '...",2583215
2,"{'resourceType': 'Patient', 'id': '6844384', '...",6844384
3,"{'resourceType': 'Patient', 'id': '6844385', '...",6844385
4,"{'resourceType': 'Patient', 'id': '6844388', '...",6844388
...,...,...
555,"{'resourceType': 'Patient', 'id': '6841240', '...",6841240
556,"{'resourceType': 'Patient', 'id': '6841241', '...",6841241
557,"{'resourceType': 'Patient', 'id': '6842052', '...",6842052
558,"{'resourceType': 'Patient', 'id': '6842064', '...",6842064


In [30]:
pack.getPatients(
	searchParams={"family":"Betterhalf"}
).shape

(12, 2)

In [31]:
multiPatientResource = pack.getResources(
    ['Patient/1849165',
     'Patient/647487']
)

multiPatientResource

,data,Patient
0,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165
1,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Serialization and Element Access

In [32]:
patientResource = patientReference.data.iloc[0].to_resource()

patientResource['name'][0]['given']

patientResource.serialize().get('birthDate')

['Series_Status: Not complete',
 'Evaluation_Status_1.0: Valid',
 'Evaluation_Status_2.0: Valid',
 'Evaluation_Status_3.0: Valid',
 'Evaluation_Status_4.0: Valid',
 'Evaluation_Status_5.0: Valid']

'2016-01-20'

In [33]:
multiPatientReference.summary
multiPatientReference.getResources().summary

,id,resourceType
0,1849165,Patient
1,647487,Patient


,id,name.given,name.family,birthDate,city,state,country
0,1849165,"[[Peter, James], [Jim], [Peter, James]]","[Chalmers, Windsor]",1974-12-25,None,None,None
1,647487,"[[Series_Status: Not complete, Evaluation_Stat...",[# 5 Pentacel at ≥ 4 yrs],2016-01-20,None,None,None


In [34]:
multiPatientResource[:1].pretty

[
    {
        "_birthDate": {
            "extension": [
                {
                    "url": "http://hl7.org/fhir/StructureDefinition/patient-birthTime",
                    "valueDateTime": "1974-12-25T14:35:45-05:00"
                }
            ]
        },
        "active": true,
        "address": [
            {
                "city": "PleasantVille",
                "district": "Rainbow",
                "line": [
                    "534 Erewhon St"
                ],
                "period": {
                    "start": "1974-12-25"
                },
                "postalCode": "3999",
                "state": "Vic",
                "text": "534 Erewhon St PeasantVille, Rainbow, Vic  3999",
                "type": "both",
                "use": "home"
            }
        ],
        "birthDate": "1974-12-25",
        "contact": [
            {
                "address": {
                    "city": "PleasantVille",
                    "district": "Rainbow"

In [35]:
pack.getDiagnosticReports(
    ['Patient/647487']
)[:1].keys


resourceType
id
meta
meta.versionId
meta.lastUpdated
meta.source
text
text.status
text.div
name
name.family
name.given
gender
birthDate


### DataFrame Operations

In [36]:
multiPatientReference[:1]

multiPatientReference[-1:]

multiPatientReference.values

multiPatientReference.info()

,data
0,{'reference': 'Patient/1849165'}


,data
1,{'reference': 'Patient/647487'}


array([[<SyncFHIRReference Patient/1849165>],
       [<SyncFHIRReference Patient/647487>]], dtype=object)

<class 'fhirpack.base.Frame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


### pack.base.Frame

In [37]:
fp.base.Frame( [[patientResource]], columns=['data'])

,data
0,"{'resourceType': 'Patient', 'id': '647487', 'm..."


### Extraction

### Patient Extraction

### fhirpack.extraction.getPatients

In [38]:
pack.getPatients(
	[
		'647487'
	]
)


,data,Patient
0,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


In [39]:
pack.getPatients(
	[
		'647487'
	]
).data.item().serialize()

{'resourceType': 'Patient',
 'id': '647487',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-03-20T21:38:58.974+00:00',
  'source': '#bE19PhJ3nQMhB1xy'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Series_Status: Not complete Evaluation_Status_1.0: Valid Evaluation_Status_2.0: Valid Evaluation_Status_3.0: Valid Evaluation_Status_4.0: Valid Evaluation_Status_5.0: Valid <b># 5 PENTACEL AT ≥ 4 YRS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>20 January 2016</span></td></tr></tbody></table></div>'},
 'name': [{'family': '# 5 Pentacel at ≥ 4 yrs',
   'given': ['Series_Status: Not complete',
    'Evaluation_Status_1.0: Valid',
    'Evaluation_Status_2.0: Valid',
    'Evaluation_Status_3.0: Valid',
    'Evaluation_Status_4.0: Valid',
    'Evaluation_Status_5.0: Valid']}],
 'gender': 'female',
 'birthDate': '2016-01-20'}

In [40]:
for i, r in pack.getPatients(['647487','1849165']).iterrows():
    print (f"Row {i} with Patient ID {r.data.id}")

Row 0 with Patient ID 647487
Row 1 with Patient ID 1849165


In [41]:
getPatientsAsList=pack.getPatients(['647487','1849165']).cast('list')

getPatientsAsList

[[<SyncFHIRResource Patient/647487>, '647487'],
 [<SyncFHIRResource Patient/1849165>, '1849165']]

In [42]:
getPatientsAsList

[[<SyncFHIRResource Patient/647487>, '647487'],
 [<SyncFHIRResource Patient/1849165>, '1849165']]

In [43]:
[e[1] for e in getPatientsAsList]

[e[0].id for e in getPatientsAsList]

['647487', '1849165']

['647487', '1849165']

### Patients as Operand and Extracting other Resources 

In [44]:
patients = pack.getPatients(
    ['9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
    '1849165',
    '649231',
    '650224']
)


### fhirpack.extraction.getRootPatient

In [45]:
patients.getRootPatients()

,data,RootPatient,Patient
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995
1,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165,1849165
2,"{'resourceType': 'Patient', 'id': '649231', 'm...",649231,649231
3,"{'resourceType': 'Patient', 'id': '650224', 'm...",650224,650224


In [46]:
pack.getRootPatients(
    [
        'b558da74-7756-4845-87d5-d1cca8b79a62',
    ],
).getLinkedPatients()

# .gatherSimplePaths(["id"]).id.unique()

,data,LinkedPatient,RootPatient,Patient
0,"{'resourceType': 'Patient', 'id': 'b558da74-77...",b558da74-7756-4845-87d5-d1cca8b79a62,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,b558da74-7756-4845-87d5-d1cca8b79a62


In [47]:
pack.getResources(
	[
		'Patient/b558da74-7756-4845-87d5-d1cca8b79a62'
	]
).getRootPatients()

,data,RootPatient,Patient
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,b558da74-7756-4845-87d5-d1cca8b79a62


In [48]:
rootPatients = patients.getRootPatients()

rootPatients.gatherSimplePaths(['id','name.given','name.family','birthDate'])

,id,name.given,name.family,birthDate
0,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,[[TestGiven]],[TestFamily],1981-01-01
1,1849165,"[[Peter, James], [Jim], [Peter, James]]","[Chalmers, Windsor]",1974-12-25
2,649231,"[[Series_Status: Complete, Evaluation_Status_1...",[Dose # 1 PCV 13 at age 12 mos # 2 at 1 yr 8 w...,2018-12-21
3,650224,"[[Series_Status: Not complete, Evaluation_Stat...",[# 1 at age ≥ 13 years old],2007-02-08


In [49]:
rootPatients.data[0].name

[{'family': 'TestFamily', 'given': ['TestGiven']}]

In [50]:
for link in rootPatients.data[0]['link']:
	lpat=link.other.to_resource()
	print(lpat.id)

b558da74-7756-4845-87d5-d1cca8b79a62


### fhirpack.extraction.getLinkedPatients

In [51]:
pack.getLinkedPatients(
	[
		'9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
	]
)

,data,LinkedPatient,RootPatient,Patient
0,"{'resourceType': 'Patient', 'id': 'b558da74-77...",b558da74-7756-4845-87d5-d1cca8b79a62,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995


### fhirpack.extraction.getConditions

### Conditions

In [52]:
conditions = pack.getConditions(
	searchParams={
		'code':'44465007',
		'_sort': '-onset-date',
	}
)[:5]

conditions

,data,Condition
0,"{'resourceType': 'Condition', 'id': '1584692',...",1584692
1,"{'resourceType': 'Condition', 'id': '1591180',...",1591180
2,"{'resourceType': 'Condition', 'id': '1402386',...",1402386
3,"{'resourceType': 'Condition', 'id': '1403562',...",1403562
4,"{'resourceType': 'Condition', 'id': '1495528',...",1495528


### Patients for Conditions

In [53]:
conditions.getPatients().explode().gatherSimplePaths(['id','name.given','name.family'])


,id,name.given,name.family
0,None,None,None
1,None,None,None
2,None,None,None
3,None,None,None
4,None,None,None
...,...,...,...
65,None,None,None
66,None,None,None
67,None,None,None
68,None,None,None


In [54]:
conditions\
	.getPatients()\
			.data.apply(lambda x:x.id)\
				.to_csv(f"data.ignore")

In [55]:
!cat data.ignore

,data
0,1402076
1,1403421
2,1495412
3,1583909
4,1590885


In [56]:
conditions = pd.read_csv( f"data.ignore", index_col=0)
conditions.shape
conditions

(5, 1)

,data
0,1402076
1,1403421
2,1495412
3,1583909
4,1590885


### Conditions for Patient

In [93]:
pack.getPatients(['8cbf1128-3644-47a1-9cc8-05f1aac6071d']).getConditions()

,data,Condition,Patient
0,"{'resourceType': 'Condition', 'id': '2b7a4d69-...",2b7a4d69-00fd-4574-86da-bdf46b21b72b,8cbf1128-3644-47a1-9cc8-05f1aac6071d
1,"{'resourceType': 'Condition', 'id': '0fc73e7c-...",0fc73e7c-18a8-47d8-bdc9-82df7f06e4a8,8cbf1128-3644-47a1-9cc8-05f1aac6071d
2,"{'resourceType': 'Condition', 'id': 'cb1bab6c-...",cb1bab6c-581b-47f4-8036-429462224f55,8cbf1128-3644-47a1-9cc8-05f1aac6071d
3,"{'resourceType': 'Condition', 'id': 'bc301bbf-...",bc301bbf-d354-4422-abe8-f7fe9c48d83c,8cbf1128-3644-47a1-9cc8-05f1aac6071d
4,"{'resourceType': 'Condition', 'id': '5db4866a-...",5db4866a-c48a-4b62-923b-96daa243226b,8cbf1128-3644-47a1-9cc8-05f1aac6071d


### fhipack.extraction.getEpisodesOfCare

In [58]:
rootPatients[:2]

,data,RootPatient,Patient
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995
1,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165,1849165


In [59]:
pack.getPatients(['P0522-patientBSJ1']).getEpisodesOfCare()

,data,EpisodeOfCare,Patient
0,"{'resourceType': 'EpisodeOfCare', 'id': 'P0522...",P0522-eLTSS-episode-Initial,None


### fhipack.extraction.getFamilyMemberHistories

In [60]:
pack.getPatients(['Patient/2866670']).getFamilyMemberHistories()

,data,FamilyMemberHistory,Patient


### fhipack.extraction.getMedicationAdministrations

In [61]:
pack.getPatients(['Patient/31678']).getMedicationAdministrations()[:5]

,data,MedicationAdministration,Patient
0,"{'resourceType': 'MedicationAdministration', '...",31675,31678


### fhipack.extraction.getMedicationRequests

In [62]:
pack.getPatients(['Patient/202205uscore-patient-example-1']).getMedicationRequests().explode()

,data,MedicationRequest,Patient


### fhipack.extraction.getObservations

In [63]:
pack\
	.getPatients(['Patient/258974'])\
		.getConditions(searchParams={"code":"44465007"})\
				.getPatients(searchParams={"family":"Keebler762"})

,data,Patient,Condition
0,"{'resourceType': 'Patient', 'id': '258974', 'm...",258974,259094


In [64]:

from datetime import datetime

interesting = pack.getPatients(['Patient/2164033']).\
    getObservations(
        searchParams={
            "code":"http ://loinc.org|29463-7",
        })\
        .gatherSimplePaths(['id','issued','code.coding.code'])\
        .explode(['code.coding.code'])

interesting.issued = pd.to_datetime(interesting.issued).dropna().apply(lambda x:x.replace(tzinfo=None))
interesting[interesting.issued> datetime.fromisoformat('2021-01-01T00:00:00')]

,id,issued,code.coding.code
0,2164039,2021-06-05 05:30:23.291,29463-7


### fhirpack.extraction.getDiagnosticReports

In [65]:
diagnosticReports=pack\
	.getPatients(["Patient/9b8c1901-62ee-48a7-8229-b771d59f1e5f"])\
		.getDiagnosticReports(
			searchParams={}
			)\

diagnosticReports

,data,DiagnosticReport,Patient
0,"{'resourceType': 'DiagnosticReport', 'id': '12...",122da94f-d24b-47ef-8409-58eb023cfb78,9b8c1901-62ee-48a7-8229-b771d59f1e5f
1,"{'resourceType': 'DiagnosticReport', 'id': '5a...",5a1a396b-783c-495c-ace9-cedce2f0c335,9b8c1901-62ee-48a7-8229-b771d59f1e5f
2,"{'resourceType': 'DiagnosticReport', 'id': '2d...",2d363733-58cb-4aba-b73f-487181359418,9b8c1901-62ee-48a7-8229-b771d59f1e5f
3,"{'resourceType': 'DiagnosticReport', 'id': '01...",013d71cd-e38e-497c-9382-b21fb5eb5f53,9b8c1901-62ee-48a7-8229-b771d59f1e5f
4,"{'resourceType': 'DiagnosticReport', 'id': '47...",471713d6-69eb-4fcf-900d-2eccf9d10f69,9b8c1901-62ee-48a7-8229-b771d59f1e5f
5,"{'resourceType': 'DiagnosticReport', 'id': 'b1...",b17c7848-f6c5-49a7-bd0a-f018ecc422de,9b8c1901-62ee-48a7-8229-b771d59f1e5f
6,"{'resourceType': 'DiagnosticReport', 'id': '58...",58c69ea7-b4be-40a5-a59e-459b8fed660a,9b8c1901-62ee-48a7-8229-b771d59f1e5f
7,"{'resourceType': 'DiagnosticReport', 'id': '87...",873433a5-68ed-42da-a6a0-d0ef3f7fd9ef,9b8c1901-62ee-48a7-8229-b771d59f1e5f
8,"{'resourceType': 'DiagnosticReport', 'id': '8c...",8c5adc06-2c8c-431f-9492-d7c9ef99635d,9b8c1901-62ee-48a7-8229-b771d59f1e5f
9,"{'resourceType': 'DiagnosticReport', 'id': 'ac...",ac83cf63-9ea8-43ce-89a8-21a6b3d3319d,9b8c1901-62ee-48a7-8229-b771d59f1e5f


In [66]:
paths=[
	"subject",
	"presentedForm.contentType",
	"presentedForm.data",
	"presentedForm.url",
	"presentedForm.title",
	"presentedForm.creation"
]



diagnosticReports=pack.getDiagnosticReports(
	searchParams={
		"_id":"19bdf90a-8ca4-4921-8aeb-2bf3423aaf09",
		# "identifier":"|",
		# "_content":"covid19",
		# "code":"|",
		# "issued__gt":"2010-01-01",
		# "issued__lt":"2011-01-01"
	}
)

diagnosticReports.gatherSimplePaths(paths)

,subject,presentedForm.contentType,presentedForm.data,presentedForm.url,presentedForm.title,presentedForm.creation
0,{'reference': 'Patient/e92abcdc-a300-41e0-aa0f...,[text/plain],[Q2pJd01UQXRNVEV0TVRNS0NpTWdRMmhwWldZZ1EyOXRjR...,None,None,None


### fhirpack.extraction.getURLBytes

In [67]:
paths=[
    'id',
    'subject.reference',
    'presentedForm.url'
]

diagnosticReports = pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
    )\
        .getDiagnosticReports()[:10]

diagnosticReports=diagnosticReports.gatherSimplePaths(paths)
diagnosticReports=diagnosticReports.explode('presentedForm.url')
diagnosticReports

# diagnosticReports['path']=diagnosticReports['id']+'_'+diagnosticReports['presentedForm.url'].str.split('/').str[-1:].str[0]
# diagnosticReports['data']=diagnosticReports.getURLBytes(operateOnCol='presentedForm.url')
# diagnosticReports.sendBytesToFile()

,id,subject.reference,presentedForm.url
0,1ca4c411-116d-49d7-82e0-cde6bff90cfd,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
1,126df183-9908-4df3-80a3-128a10fd6b3a,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
2,19bdf90a-8ca4-4921-8aeb-2bf3423aaf09,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
3,71701d6e-3bba-44c0-af75-271558aedb22,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
4,540317ac-5490-4186-8930-8bfbdaad1055,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
5,97f7f1b1-661f-4076-9352-e698cdf9ef6f,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
6,9995355f-1361-45de-aebb-a7446f8f13f7,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
7,34c775ba-b76d-4ef0-a69f-ab88e90ad151,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
8,1ff86952-9480-45b5-953d-e6056f74659f,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
9,5bd0f37e-7c1f-40ac-b654-97fa66875f70,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None


In [68]:
diagnosticReports.loc[:, diagnosticReports.columns!='data'][:5]

,id,subject.reference,presentedForm.url
0,1ca4c411-116d-49d7-82e0-cde6bff90cfd,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
1,126df183-9908-4df3-80a3-128a10fd6b3a,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
2,19bdf90a-8ca4-4921-8aeb-2bf3423aaf09,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
3,71701d6e-3bba-44c0-af75-271558aedb22,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
4,540317ac-5490-4186-8930-8bfbdaad1055,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None


### fhirpack.extraction.base.getAbsolutePaths

In [69]:
absolutePaths=[
#    'Condition.code.coding.code',
   'Condition.id',
    'Condition.subject.reference',
    'Encounter.id',
    'Encounter.participant.individual.reference',
#     'Procedure.code.coding.display',
#     'Procedure.code.coding.code',
    'Procedure.id',
    'Procedure.status',
    'Procedure.performedDateTime'
]

pack.client

result=pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
).getAbsolutePaths(absolutePaths)

<SyncFHIRClient https://hapi.fhir.org/baseR4>

In [70]:
result['Procedure']

,Procedure.id,Procedure.performedDateTime,Procedure.status
0,f262bafb-789a-44a1-9453-c21065abc4d1,None,completed
1,e65c5378-5d1a-4978-9590-c6c8846dd6d0,None,completed


In [71]:
result["Condition"]

,Condition.id,Condition.subject.reference
0,b8a1a1c4-f973-4b6b-a0ea-3d04bc4c8823,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc


In [72]:
result["Encounter"]

,Encounter.id,Encounter.participant.individual.reference
0,b9f19065-c8e6-40b9-b368-b09a8fb9a8d6,[Practitioner/00000171-0929-2892-0000-00000001...
1,174d8c79-5517-4f80-967f-3939f943b2c8,[Practitioner/00000171-0929-2892-0000-00000001...
2,3f3a061a-5df2-4dc0-872c-e1e4b88955a1,[Practitioner/00000171-0929-2892-0000-00000001...
3,2b8cb0e7-5b1e-4769-a418-bf9e6dd86611,[Practitioner/00000171-0929-2892-0000-00000001...
4,36fb900c-9ff7-4131-812c-536ab50a0a5c,[Practitioner/00000171-0929-2892-0000-00000000...
5,9911ef15-1b9a-4fbd-b95d-05bd4f060269,[Practitioner/00000171-0929-2892-0000-00000001...
6,f3b1e501-aac6-4480-bf35-38474563069a,[Practitioner/00000171-0929-2892-0000-00000001...
7,969f9166-4f9a-4be6-9257-ed00b295a408,[Practitioner/00000171-0929-2892-0000-00000001...
8,806264f4-4f45-4105-a5bc-8e9a9c49b119,[Practitioner/00000171-0929-2892-0000-00000001...
9,40166ee5-0bc3-4819-96c9-e07dd26e938b,[Practitioner/00000171-0929-2892-0000-00000001...


In [73]:
result['Encounter'].explode('Encounter.participant.individual.reference')\
	.rename(columns={"Encounter.participant.individual.reference":"data"})\
			.dropna()\
				.getResources()[:5]\
					.gatherSimplePaths(['name.given','name.family'])

,name.given,name.family
0,[[Willy639]],[Rutherford999]
1,[[Lilla884]],[Mills423]


In [74]:
paths=[
	'valueQuantity.value',
	'valueQuantity.unit',
	'valueQuantity.system',
	'valueQuantity.code',
]

In [75]:
pack.getPatients(['1555106'])\
		.getObservations()[:10]\
				.gatherSimplePaths(paths)

,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code
0,102.12000,U/L,http://unitsofmeasure.org,U/L
1,7.58040,g/dL,http://unitsofmeasure.org,g/dL
2,6.58630,mL/min,http://unitsofmeasure.org,mL/min
3,31.21100,pg,http://unitsofmeasure.org,pg
4,20.88000,mmol/L,http://unitsofmeasure.org,mmol/L
5,94.10200,fL,http://unitsofmeasure.org,fL
6,32.47200,g/dL,http://unitsofmeasure.org,g/dL
7,87.16000,mg/dL,http://unitsofmeasure.org,mg/dL
8,78.92000,%,http://unitsofmeasure.org,%
9,0.27443,10*3/uL,http://unitsofmeasure.org,10*3/uL


### fhirpack.extraction.getImagingStudies

In [92]:
conds=pack.getConditions(
	searchParams={
		"code":"C61",
		# "_content": "leber"
	}
	)
conds

,data,Condition
0,"{'resourceType': 'Condition', 'id': '1849657',...",1849657


In [77]:
pack\
	.getImagingStudies(searchParams={"endpoint:missing":False})\
		.gatherSimplePaths(['series.description','series.uid','identifier.value'])

# use .getDICOMInstances().sendDICOMToFiles() to download DICOM files

,series.description,series.uid,identifier.value
0,None,[IdType-2],[Identifier-117]
1,None,[IdType-5],[Identifier-118]
2,None,None,None
3,None,[1.2.276.0.7230010.3.1.3.8323329.10.1594989066...,[urn:oid:1.2.826.0.1.3680043.8.498.94636677472...


### Transformation

### fhirpack.transformation.base.gatherKeys

In [78]:
patients.gatherKeys().data.value_counts()[:10]

[resourceType, id, meta, meta.versionId, meta.lastUpdated, meta.source, text, text.status, text.div, name, name.text, name.family, name.given, gender, birthDate]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [79]:
pack.getPatients(['e92abcdc-a300-41e0-aa0f-378daebe25dc'])\
        .gatherKeys(['valueString'])

,data
0,"[resourceType, id, meta, meta.versionId, meta...."


### fhirpack.transformation.base.valuesForKeys

In [80]:
pack.getPatients(['e92abcdc-a300-41e0-aa0f-378daebe25dc'])\
        .gatherValuesForKeys(['valueString'])

,data
0,"[White, Not Hispanic or Latino, Nanci249 Gleic..."


### fhirpack.transformation.base.gatherReferences

In [81]:
pack.gatherReferences(
    ['Patient/9ac622c4-61c8-40b4-b6d9-06cfeb3d6995']
    , recursive=True
)

,referencer,referencee
0,{'reference': 'Patient/9ac622c4-61c8-40b4-b6d9...,[Patient/b558da74-7756-4845-87d5-d1cca8b79a62]
1,{'reference': 'Patient/b558da74-7756-4845-87d5...,[]


### fhirpack.transformation.base.gatherText

In [82]:
pack.getPatients(['Patient/1555106'],)\
        .getObservations()[:5]\
                .gatherText()

,data
0,"[laboratory, 102.12, Alkaline phosphatase [Enz..."
1,"[Protein [Mass/volume] in Serum or Plasma, lab..."
2,[Glomerular filtration rate/1.73 sq M.predicte...
3,"[MCH [Entitic mass] by Automated count, labora..."
4,"[20.88, Carbon dioxide, total [Moles/volume] i..."


### fhirpack.transformation.base.gatherDates

In [83]:
pack.getPatients(['Patient/1555106'],)\
        .getObservations()[:5]\
                .gatherValuesForKeys(['issued'])

,data
0,[2020-03-18T21:53:14.197-04:00]
1,[2020-03-19T21:53:14.197-04:00]
2,[2020-03-19T21:53:14.197-04:00]
3,[2020-03-14T21:53:14.197-04:00]
4,[2020-03-19T21:53:14.197-04:00]


In [94]:
pack.getPatients(['Patient/1555106'],)\
        .getObservations()[:5]\
                .gatherDates()

,dates
0,"[2020-03-18T21:53:14-04:00, 2020-03-18T21:53:1..."
1,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."
2,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."
3,"[2020-03-14T21:53:14-04:00, 2020-03-14T21:53:1..."
4,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."


### Load

In [ ]:
pack.validate(['Patient/1555106']).sendResourcesToFiles(['./data.ignore'])

array([ True])

### Utils

### Server Profiling

In [ ]:
pack.countServerResources()

,resourceType,count
0,Account,510
1,ActivityDefinition,284
2,AdverseEvent,116
3,AllergyIntolerance,9356
4,Appointment,29636
...,...,...
141,TestReport,11
142,TestScript,33
143,ValueSet,3126
144,VerificationResult,9


### Validation

In [ ]:
pack.validate(['Patient/1555106'])

,data
0,"{'resourceType': 'Patient', 'id': '1555106', '..."


### FHIR PACK Internals

#### FHIRPy

### URL Parsing

In [ ]:
import fhirpy.base.utils as fpu
url=fpu.parse_pagination_url('/app/FHIR/r4/Condition?code=C61&_count=1&_sort=-onset-date&identifier=%7C&_Pagination=eyJvZmZzZXQiOjIwfQ%3D%3D')
url

('/app/FHIR/r4/Condition',
 {'code': ['C61'],
  '_count': ['1'],
  '_sort': ['-onset-date'],
  'identifier': ['|'],
  '_Pagination': ['eyJvZmZzZXQiOjIwfQ==']})

### Raw fetch_resource()

In [ ]:
pack.client._fetch_resource(
	"Condition/1499041"
)

{'resourceType': 'Condition',
 'id': '1499041',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-10-05T16:29:07.960+00:00',
  'source': '#0sjLd8ZeFHHrRHaX'},
 'clinicalStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical',
    'code': 'active'}]},
 'verificationStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-ver-status',
    'code': 'confirmed'}]},
 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-category',
     'code': 'encounter-diagnosis',
     'display': 'Encounter Diagnosis'}]}],
 'code': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '22298006',
    'display': 'Myocardial Infarction'}],
  'text': 'Myocardial Infarction'},
 'subject': {'reference': 'Patient/1465609'},
 'onsetDateTime': '2020-04-01T09:52:49-04:00',
 'recordedDate': '2020-04-01T09:52:49-04:00',
 'asserter': {'reference': 'Practitioner/1498740'}}

### Raw client.execute()

In [ ]:
pack.client.execute('Condition',
method='get',
params={
	'code':'C61',
	'_sort':'-onset-date',
	'_count': 1,
	'_total':'accurate'
}
).total

pack.client.execute(
	'DiagnosticReport/_search',
	method="post",
	data={"_id": "9fa9ab4c-9e1c-46c9-9bec-ec01b3215716"},
	params={"_count": 1},
)

1

{'resourceType': 'Bundle',
 'id': 'af8b2454-29bd-4fe3-b350-733131e9383a',
 'meta': {'lastUpdated': '2022-07-13T08:04:01.285+00:00'},
 'type': 'searchset',
 'link': [{'relation': 'self',
   'url': 'https://hapi.fhir.org/baseR4/DiagnosticReport/_search'},
  {'relation': 'next',
   'url': 'https://hapi.fhir.org/baseR4?_getpages=af8b2454-29bd-4fe3-b350-733131e9383a&_getpagesoffset=1&_count=1&_pretty=true&_bundletype=searchset'}],
 'entry': [{'fullUrl': 'https://hapi.fhir.org/baseR4/DiagnosticReport/6630582',
   'resource': {'resourceType': 'DiagnosticReport',
    'id': '6630582',
    'meta': {'versionId': '1',
     'lastUpdated': '2022-07-05T06:03:55.922+00:00',
     'source': '#VSn1gQals3CWesRQ'},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"> Untitled Diagnostic Report </div><table class="hapiPropertyTable"><tbody><tr><td>Status</td><td>REGISTERED</td></tr><tr><td>Issued</td><td> 15 May 2013 18:32:52 </td></tr><tr><

### Customization

### fhirpack.custom.extraction.base

In [ ]:
# the fhirpack.custom package is guaranteed to never be used by us
# use it to customize fhirpack

# pack.unimplementedPluginBaseExtractorMethod()
# pack.unimplementedPluginBaseTransformerMethod()
# pack.unimplementedPluginBaseLoaderMethod()

# pack.unimplementedPluginSampleExtractorMethod()
# pack.unimplementedPluginSampleTransformerMethod()
# pack.unimplementedPluginSampleLoaderMethod()